In [ ]:
from keras.applications.resnet50 import ResNet50
# resnet_weights = '../input/resnet50/resnet50_weigts_tf_dim_ordering_tf_kernels.h5'
resnet_model = ResNet50(weights='imagenet', include_top=False )

# _get_predictions(resnet_model)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from PIL import Image


In [ ]:
input_path = "/content/drive/My Drive/LIDC-IDRI-0001/01-01-2000-30178/3000566-03192/"
# train and valid = flowr_path
train = input_path+"/Training/"
valid = input_path+"/Validation/"
test = input_path+"/Testing"

In [ ]:
def _get_features(img_path):
  img = image.load_img(img_path, target_size=(224,224))
  img_data = image.img_to_array(img)
  img_data = np.expand_dims(img_data, axis=0)
  img_data = preprocess_input(img_data)
  resnet_features = resnet_model.predict(img_data)
  return resnet_features


In [ ]:
classes = os.listdir(train)
print("No of classes: ", len(classes))
print("Class Labels: ", classes)

No of classes:  2
Class Labels:  ['Positive', 'Negative']


In [ ]:
# import pandas as pd
# train_list = []

# # images = species_num, labels = species, train_list = train, train = train_dir
# for images, labels in enumerate(classes):
#   for file in os.listdir(os.path.join(train+ labels)):
#       train_list.append(['/content/drive/My Drive/LIDC-IDRI-0001/01-01-2000-30178/3000566-03192/Training/{}/{}'.format(labels, file), images, labels])
        
# train_list = pd.DataFrame(train_list, columns=['file', 'images', 'labels'])

# print('Training Data: ',train_list.shape)

In [ ]:
# features = []
# for i in range(len(train_list)):
#   features.append(_get_features(train_list['file'][i]))

In [ ]:
datagen = ImageDataGenerator (rescale=1./255,
                              rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
batch_size = 20

Found 962 images belonging to 2 classes.
Found 206 images belonging to 2 classes.


In [ ]:
def extract_features(directory, sample_count):
  feats = np.zeros(shape=(sample_count,7,7,2048))
  lab = np.zeros(shape=(sample_count,2))
  generator = datagen.flow_from_directory(
        directory,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')

  
  i =0
  for inputs_batch, label_batch in generator:
    features_batch = resnet_model.predict(inputs_batch)
    feats[i * batch_size:(i+1)*batch_size] = features_batch
    lab[i*batch_size:(i+1)*batch_size] = label_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return feats,lab

In [ ]:
train_feats, train_labs= extract_features(train,962)
valid_feats, valid_labs= extract_features(valid,206)
test_feats, test_labs = extract_features(test,208)

Found 962 images belonging to 2 classes.
Found 206 images belonging to 2 classes.
Found 208 images belonging to 2 classes.


In [ ]:
train_feats = np.reshape(train_feats,(962,7*7*2048))
valid_feats = np.reshape(valid_feats,(206,7*7*2048))
test_feats = np.reshape(test_feats, (208,7*7*2048))

In [ ]:
# from keras.applications import VGG16
# clf = VGG16()

553467904/553467096 [==============================] - 4s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras import metrics


In [ ]:
from keras import models, layers, optimizers
from tensorflow.keras.layers import Reshape, Flatten

model = models.Sequential()
model.add(layers.Dense(512,activation='relu',input_dim=7*7*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation='relu',input_dim=7*7*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation='relu',input_dim=7*7*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation='relu',input_dim=7*7*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation='relu',input_dim=7*7*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', metrics.SensitivityAtSpecificity()])

TypeError: ignored

In [ ]:
# print(train_feats.shape,train_feats.ndim,train_feats.dtype)
# print(train_labs.shape,train_labs.ndim,train_labs.dtype)
# print(valid_feats.shape,valid_feats.ndim,valid_feats.dtype)
# print(valid_labs.shape,valid_labs.ndim)

(962, 7, 7, 2048) 4 float64
(962, 2) 2 float64
(206, 7, 7, 2048) 4 float64
(206, 2) 2


In [ ]:
model.fit(train_feats, train_labs, epochs=30, validation_data=(valid_feats,valid_labs))

Epoch 1/30
31/31 [==============================] - 1s 30ms/step - loss: 0.6950 - accuracy: 0.4823 - specificity_at_sensitivity_2: 0.4314 - val_loss: 0.6940 - val_accuracy: 0.5000 - val_specificity_at_sensitivity_2: 0.5000
Epoch 2/30
31/31 [==============================] - 1s 21ms/step - loss: 0.6987 - accuracy: 0.5031 - specificity_at_sensitivity_2: 0.4761 - val_loss: 0.6939 - val_accuracy: 0.5000 - val_specificity_at_sensitivity_2: 0.5000
Epoch 3/30
31/31 [==============================] - 1s 21ms/step - loss: 0.6942 - accuracy: 0.4730 - specificity_at_sensitivity_2: 0.4023 - val_loss: 0.6933 - val_accuracy: 0.5000 - val_specificity_at_sensitivity_2: 0.5000
Epoch 4/30
31/31 [==============================] - 1s 21ms/step - loss: 0.6939 - accuracy: 0.5073 - specificity_at_sensitivity_2: 0.4657 - val_loss: 0.6935 - val_accuracy: 0.5000 - val_specificity_at_sensitivity_2: 0.5000
Epoch 5/30
31/31 [==============================] - 1s 21ms/step - loss: 0.6958 - accuracy: 0.5073 - specifi

In [ ]:
spec = tf.keras.metrics.SpecificityAtSensitivity(0.5)

In [ ]:
sens = tf.keras.metrics.SensitivityAtSpecificity(0.5)

In [ ]:
print(sens.result().numpy)

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=float32, numpy=0.0>>


Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
      train,
      target_size=(224,224),
      batch_size=batch_size,
      class_mode='categorical')
valid_generator = test_datagen.flow_from_directory(
    valid,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=valid_generator)